In [ ]:
!pip install tensorflow
!pip install keras
!pip install kaggle

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Upload Kaggle JSON
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download Datasets from Kaggle
!kaggle datasets download -d mylee2009/shopee-round-2-product-detection-challenge

100% 9.37G/9.38G [03:42<00:00, 40.2MB/s]
100% 9.38G/9.38G [03:43<00:00, 45.1MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('shopee-round-2-product-detection-challenge.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

In [ ]:
# from keras.applications import ResNet50
# import tensorflow
# from keras.models import Sequential
# from keras.layers import Dense, Flatten, GlobalAveragePooling2D

# num_classes = 42

# my_new_model = Sequential()
# my_new_model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
# my_new_model.add(Dense(num_classes, activation='softmax'))

# # Say not to train first layer (ResNet) model. It is already trained
# my_new_model.layers[0].trainable = False

In [ ]:
import os
import cv2
import numpy as np
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
import tensorflow as tf
import pandas as pd


train_path = '/content/files/train/train';
test_path = '/content/files/test'

def get_files(path, save_data):
	for i in os.listdir(path):
		if(os.path.isdir(path + "/" + i)):
			get_files(path + "/" + i, save_data)
		else:
			save_data.append(path + "/" + i)

In [ ]:
df = pd.read_csv('/content/files/train.csv')
result = []
for i in range(len(df)):
  if df.category[i]<10:
    result.append('0'+str(df.category[i]))
  else:
    result.append(str(df.category[i]))

df.category = result
df.head()

,filename,category
0,45e2d0c97f7bdf8cbf3594beb6fdcda0.jpg,03
1,f74d1a5fc2498bbbfa045c74e3cc333e.jpg,03
2,f6c172096818c5fab10ecae722840798.jpg,03
3,251ffd610399ac00fea7709c642676ee.jpg,03
4,73c7328b8eda399199fdedec6e4badaf.jpg,03


In [ ]:
!pip install imgaug

In [ ]:
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

  Cloning https://github.com/mjkvaak/ImageDataAugmentor to /tmp/pip-req-build-qg3stqg2
  Running command git clone -q https://github.com/mjkvaak/ImageDataAugmentor /tmp/pip-req-build-qg3stqg2
     |████████████████████████████████| 28.2MB 101kB/s 
  Created wheel for ImageDataAugmentor: filename=ImageDataAugmentor-0.0.0-cp36-none-any.whl size=28017 sha256=edeeb849c4eed8ca0a5154d12fced8ad06546cfadb781e5f77c7b83fa1d8cac7
  Stored in directory: /tmp/pip-ephem-wheel-cache-9fey4ws9/wheels/d9/10/55/6fca35a4072f87d694876d56ece64db3846cf45e1da1c381fe
Successfully built ImageDataAugmentor
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [ ]:
from ImageDataAugmentor.image_data_augmentor import *
from imgaug import augmenters as iaa
import imgaug as ia


In [ ]:
def load_data():
    """This function downloads, extracts, loads, normalizes and one-hot encodes Flower Photos dataset"""
    # get all classes for this dataset (types of flowers) excluding LICENSE file
    # roses = list(data_dir.glob('roses/*'))
    # 20% validation set 80% training set
    image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)
    # make the training dataset generator
    train_data_gen = image_generator.flow_from_directory(directory=train_path + "/", batch_size=batch_size,
                                                        target_size=(IMAGE_SHAPE[0], IMAGE_SHAPE[1]),
                                                        shuffle=True, subset="training")
    # make the validation dataset generator
    test_data_gen = image_generator.flow_from_directory(directory=train_path + "/", batch_size=batch_size, 
                                                        target_size=(IMAGE_SHAPE[0], IMAGE_SHAPE[1]),
                                                        shuffle=True, subset="validation")
    return train_data_gen, test_data_gen

In [ ]:
def create_model(input_shape):
    # load MobileNetV2
    model = ResNet152V2(input_shape=input_shape)
    # remove the last fully connected layer
    model.layers.pop()
    # freeze all the weights of the model except the last 4 layers
    for layer in model.layers[:-4]:
        layer.trainable = False
    # construct our own fully connected layer for classification
    output = Dense(42, activation="softmax")
    # connect that dense layer to the model
    output = output(model.layers[-1].output)
    model = Model(inputs=model.inputs, outputs=output)
    # print the summary of the model architecture
    model.summary()
    # training the model using rmsprop optimizer
    model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
    return model

In [ ]:
from keras.applications import MobileNetV2, DenseNet201, ResNet152V2
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.layers import Dense
IMAGE_SHAPE = (224, 224, 3)
epochs = 3
batch_size = 42
train_generator, validation_generator = load_data()
model = create_model(input_shape=(IMAGE_SHAPE))
# model name
# model_name = "MobileNetV2_finetune_last5"
# some nice callbacks
# tensorboard = TensorBoard(log_dir=f"logs/{model_name}")
# checkpoint = ModelCheckpoint(f"results/{model_name}" + "-loss-{val_loss:.2f}-acc-{val_acc:.2f}.h5",
#                             save_best_only=True,
#                             verbose=1)
# make sure results folder exist
if not os.path.isdir("results"):
    os.mkdir("results")

# count number of steps per epoch
training_steps_per_epoch = np.ceil(train_generator.samples / batch_size)
validation_steps_per_epoch = np.ceil(validation_generator.samples / batch_size)

Found 84331 images belonging to 42 classes.
Found 21061 images belonging to 42 classes.
242753536/242745792 [==============================] - 8s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (Non

In [ ]:
# train using the generators
from keras.callbacks import ReduceLROnPlateau
model.load_weights("/content/drive/My Drive/Colab Notebooks/Model Product Detection/model_resnet152V2v1.h5")
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001)
model.fit_generator(train_generator, steps_per_epoch=training_steps_per_epoch,
                    validation_data=validation_generator, validation_steps=validation_steps_per_epoch//3,
                    epochs=2*epochs, verbose=1, callbacks=[reduce_lr])

Epoch 1/6
2008/2008 [==============================] - 1187s 591ms/step - loss: 1.3153 - accuracy: 0.6480 - val_loss: 1.3250 - val_accuracy: 0.6359
Epoch 2/6
 588/2008 [=======>......................] - ETA: 12:31 - loss: 1.2920 - accuracy: 0.6525

KeyboardInterrupt: ignored

In [ ]:
model.save_weights("drive/My Drive/Colab Notebooks/Model Product Detection/model_resnet152V2v1.h5")

In [ ]:
from keras.preprocessing import image
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224),grayscale=False)
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [ ]:
result_set = pd.read_csv('/content/files/test.csv')
image_name = result_set.filename
result = []
for name in image_name:
  pred = model.predict(load_image(test_path+"/test/"+name))
  y_pred = [np.argmax(probas) for probas in pred]
  result.append(str(y_pred[0]).zfill(2))
result_set.category = result

In [ ]:
from google.colab import files
result_set.to_csv('result_ResNet152V2_v1.csv',index=False) 
files.download('result_ResNet152V2_v1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>